In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
# Additional libraries for data handling, preprocessing, etc.


In [2]:
# Define input shape based on your image size (here, 640x360)
input_shape = (360, 640, 3)

# Load MobileNet, excluding the final classification layer
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the pre-trained layers to avoid retraining them (optional)
for layer in mobilenet.layers:
  layer.trainable = False


C:\Users\ukrst\AppData\Local\Temp\ipykernel_1104\1475731864.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)


In [3]:
# GlobalAveragePooling2D reduces spatial dimensions to a single vector
x = GlobalAveragePooling2D()(mobilenet.output)

# Dense layer with 4 units (one for each variant) and softmax activation
predictions = Dense(2, activation='sigmoid')(x)

# Create the final model
model = tf.keras.Model(inputs=mobilenet.input, outputs=predictions)


In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
import cv2
import os
import numpy as np

# Define empty lists for images and labels
X = []
y = []

# Folder path (replace 'path/to/images' with your actual folder path)
folder_path = 'classification_data/'

# Loop through folders
for folder_name in ['attack_img', 'explore_img']: #, 'loot_img', 'getaway_img']:
    folder_full_path = os.path.join(folder_path + folder_name)
    
    class_label = {
      'attack_img': [1,0],
      'explore_img': [0,1],
      # 'loot_img': [0,0,1,0],
      # 'getaway_img': [0,0,0,1]
    }[folder_name]
    
    for filename in os.listdir(folder_full_path):
        if filename.endswith(".jpg") :
            img = cv2.imread(os.path.join(folder_full_path, filename))
        
            # Append image to X and label to y
            X.append(img)
            
            
            y.append(class_label)
        

X = np.asarray(X)
y = np.asarray(y)

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
# (print(y_train[-100:-1]))

(444, 360, 640, 3) (444, 2)


In [12]:
model.fit(X_train, y_train, epochs=80, validation_data=(X_val, y_val))


Epoch 1/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.4643 - loss: 1.3438 - val_accuracy: 0.7297 - val_loss: 0.7436
Epoch 2/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.6708 - loss: 0.7405 - val_accuracy: 0.7207 - val_loss: 0.5860
Epoch 3/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7327 - loss: 0.5906 - val_accuracy: 0.7477 - val_loss: 0.5305
Epoch 4/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7245 - loss: 0.5477 - val_accuracy: 0.7477 - val_loss: 0.5073
Epoch 5/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7091 - loss: 0.5116 - val_accuracy: 0.7297 - val_loss: 0.4857
Epoch 6/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.7392 - loss: 0.4987 - val_accuracy: 0.7387 - val_loss: 0.4728
Epoch 7/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7591 - loss: 0.4786 - val_accuracy: 0.7387 - val_loss: 0.4595
Epoch 8/80
14/14 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.7886 - loss: 0.4453 - val_accuracy: 0.7658 - val_loss:

In [13]:
loss, accuracy = model.evaluate(X_val, y_val)
print('Test accuracy:', accuracy)

model.save('classificator_2.h5')

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8008 - loss: 0.4113


Test accuracy: 0.8198198080062866


In [14]:
model_rap = tf.keras.models.load_model('classificator_2.h5')


for n in range(20):
    y_real = y_val[n]
    tbp = np.asarray(X_val[n])
    cv2.imshow('ss', tbp)
    tbp = np.expand_dims(tbp, axis=0)
    predictions = model_rap.predict(tbp)
    print(predictions, "\n", y_real)

    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 792ms/step
[[0.6606835  0.25001004]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
[[0.70638585 0.16150951]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[0.8504597  0.12777892]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.5922915 0.268165 ]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.88185024 0.06126564]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[0.498731   0.31036833]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
[[0.7195252  0.10257636]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
[[0.8738252 0.0411672]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.6407304  0.12588255]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
[[0.49109855 0.38159853]] 
 [0 1]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
[[0.85450214 0.12684175]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
[[0.793506   0.14177214]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.47418916 0.17560378]] 
 [1 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[0.77665114 0.26976922]] 
 [

In [28]:
model_rap = tf.keras.models.load_model('my_mobilenet_model.h5')
model_rap.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_rap.fit(X_train, y_train, epochs=40, validation_data=(X_val, y_val))
model_rap.save('class_2.h5')

Epoch 1/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.7854 - loss: 0.5956 - val_accuracy: 0.7400 - val_loss: 0.7340
Epoch 2/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.8160 - loss: 0.5927 - val_accuracy: 0.7000 - val_loss: 0.7350
Epoch 3/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7564 - loss: 0.6643 - val_accuracy: 0.6900 - val_loss: 0.7348
Epoch 4/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7663 - loss: 0.5880 - val_accuracy: 0.7200 - val_loss: 0.7174
Epoch 5/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7939 - loss: 0.5774 - val_accuracy: 0.7500 - val_loss: 0.6913
Epoch 6/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7758 - loss: 0.5911 - val_accuracy: 0.7700 - val_loss: 0.6796
Epoch 7/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8515 - loss: 0.5431 - val_accuracy: 0.7200 - val_loss: 0.6956
Epoch 8/40
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8156 - loss: 0.5335 - val_accuracy: 0.7600 - val_loss: